In [2]:
# !pip install langchain_community
# !pip install pymongo[srv]
# !pip install dnspython
# !pip install langchain_openai
# !pip install sentence-transformers

In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import MongoDBAtlasVectorSearch
from pymongo.mongo_client import MongoClient
from langchain.embeddings import HuggingFaceEmbeddings
import os

import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")
MONGO_DB_URI = os.getenv("MONGO_DB_URI")

In [2]:
df = pd.read_csv('../data/shopee_qna_data_clean.csv')

In [3]:
answers_texts = df['answers'].dropna().to_list()
answers_texts[0]

'Aktivitas lelang di Shopee Live memungkinkan Anda untuk memenangkan penawaran produk yang ditetapkan oleh host/Penjual.Setelah Penjual memulai lelang, Kartu Lelang akan muncul di layar Anda. Kartu ini menunjukkan jenis lelang dan penghitung waktu mundur yang sudah ditentukan oleh Penjual sebagai batas akhir Anda untuk melakukan penawaran.Untuk berpartisipasi, pilih Gabung Sekarang setelah kartu lelang berubah menjadi oranye.\r\nSetelah Lelang selesai, urutan pemenang dapat dilihat dengan memilih ikon Lelang. Nama pemenang yang dimunculkan hanya 10 orang yang tercepat.\r\n️⚠️ Catatan:Beberapa Lelang hanya terbuka untuk Pengikut toko. Ikuti toko untuk berpartisipasi.Pemenang Lelang ditetapkan berdasarkan kecepatan Penawar.Apabila Anda menjadi pemenangnya, Penjual akan menghubungi Anda secara terpisah melalui chat atau sesuai ketentuan yang berlaku dari Penjual.\r\nApakah artikel ini berguna?YaTidak'

Code dibawah memecah teks panjang (misalnya dari jawaban chatbot, artikel, dokumen, dsb) menjadi potongan-potongan kecil (chunk) agar bisa lebih mudah diproses dan di-index oleh model berbasis embedding/vector store.

In [4]:
# proses preprocessing data teks sebelum dimasukkan ke vector store

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.create_documents(answers_texts)

In [5]:
# %%capture
# proses embedding

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Membangun koneksi dengan MongoDB
# URI mongo db silahkan anda ganti dengan URI dengan URI database mongodb anda
client = MongoClient(MONGO_DB_URI, tls=True)

In [8]:
collection = client['shopee']['answer_vec']

Menyimpan dokumen yang sudah dipotong (docs) ke dalam vector store berbasis MongoDB Atlas

In [ ]:
docsearch = MongoDBAtlasVectorSearch.from_documents(
    docs, embeddings, collection=collection, index_name="vector_index"
)